In [1]:
import json
import os
import pandas as pd
from time import time
from tqdm import tqdm
from typing import List, Dict, Tuple
from src.RCH_module.RCH import get_volumes
from src.utils.utils import load_data, preprocess_data
from src.evaluation_module.evaluation import evaluate_solution

In [2]:
def select_viajes(df_viajes: pd.DataFrame) -> pd.DataFrame:
    """
    Select specific voyages based on criteria:
    - Container type is "MAXI 45'"
    - Loaded last week.
    """
    # Filter only "MaXI 45'" container type
    filter_1 = df_viajes["TipoEquipo"] == "MAXI 45'"
    filter_2 = df_viajes["FechaCierreEstancia"] >= pd.Timestamp.now() - pd.Timedelta(days=7)
    # Select relevant columns
    columns = [
        "CodigoViaje", "FechaCierreEstancia", "NumExpediciones", "Destinos","AltoCm", "LargoCm",
        "AnchoCm", "VolumenCargado", "VolumenMax", "Volumen%"
    ]

    # Filter the DataFrame based on the conditions
    df_viajes = df_viajes[filter_1 & filter_2][columns]

    # Drop viajes that have NaN in "VolumenCargado"
    df_viajes = df_viajes.dropna(subset=["VolumenCargado"])

    # Separate destinations into list
    df_viajes["Destinos"] = df_viajes["Destinos"].apply(lambda x: x.split(";"))

    return df_viajes.reset_index(drop=True)


CodigoViaje	TipoEquipo	TipoEquipoAltoMetros	TipoEquipoAnchoMetros	TipoEquipoLongitudMetros	FechaApertura	FechaCierreViaje	FechaCierreEstancia	FechaPrimeraLectura	FechaUltimaLectura	NumExpediciones	NumPartidas	NumPalets	NumBultos	Destinos
VBCN2501790	MAXI 45'	2,4	2,4	13,5	05/06/2025 14:49	05/06/2025 19:15	05/06/2025 19:15	05/06/2025 15:41	05/06/2025 19:15	34	52	47	5	PMI



In [11]:
def select_initial_partidas(df_partidas: pd.DataFrame) -> pd.DataFrame:
    """
    Select partidas that where in the warehouse at the beginning of last week.

    Args:
        df_partidas (pd.DataFrame): DataFrame containing partida information.

    Returns:
        pd.DataFrame: Filtered DataFrame with relevant partidas.
    """
    # Filter partidas that were in the warehouse at the beginning of last week
    # Select last monday at 00:00:00
    now = pd.Timestamp.now()
    last_monday = now - pd.Timedelta(days=now.weekday()+7, hours=now.hour, minutes=now.minute, seconds=now.second)
    filter_1 = df_partidas["FechaCargaContenedor"] >= last_monday
    

    # CodigoViaje	FechaCargaContenedor	FechaEntradaAlmacen	Expedicion	Partida	PesoKg	LargoCm	AltoCm	AnchoCm	TipoPartida	Remontable	Destino	DatosClonados	Adr	DtLevanteExport	PuroVolumetrica	TratadaVOlumetrica	Cliente	Volumen


    # Filter the DataFrame based on the conditions
    df_partidas = df_partidas[filter_1]
    # Drop the rows with NaN in "PesoKg", "LargoCm", "AltoCm", "AnchoCm"
    df_partidas = df_partidas.dropna(subset=["PesoKg", "LargoCm", "AltoCm", "AnchoCm"])
    # Reset index
    df_partidas = df_partidas.reset_index(drop=True)

    return df_partidas

In [4]:
def update_partidas_warehouse(df_partidas: pd.DataFrame, solucion) -> pd.DataFrame:
    """
    Update the partidas DataFrame with the volumes from the solution.

    Args:
        df_partidas (pd.DataFrame): DataFrame containing partida information.
        solucion (dict): Solution dictionary containing volumes for each partida.

    Returns:
        pd.DataFrame: Updated DataFrame with volumes.
    """
    loaded_partidas = [i for ((i, _), _) in solucion["solution"]]
    # Drop partidas that are in the solution
    df_partidas = df_partidas[~df_partidas["Partida"].isin(loaded_partidas)]

    # Reset the index
    df_partidas = df_partidas.reset_index(drop=True)

    # Return the updated DataFrame
    return df_partidas

In [5]:
def select_partidas_at_time_for_dest(df_partidas: pd.DataFrame, time: str, dest: List[str]) -> pd.DataFrame:
    """
    Select shipments that are present in the warehouse at a specific time for a given destination.

    Args:
        df_partidas (pd.DataFrame): DataFrame containing partida information.
        time (str): Time in the format "YYYY-MM-DD HH:MM:SS".
        dest (List[str]): List of destinations to filter by.

    Returns:
        pd.DataFrame: Filtered DataFrame with relevant partidas.
    """
    # Convert time to timestamp
    time = pd.Timestamp(time)

    # Filter partidas that are in the warehouse at the specified time
    filter_1 = df_partidas["FechaEntradaAlmacen"] < time
    filter_2 = df_partidas["Destino"].isin(dest)

    # Filter the DataFrame based on the conditions
    df_partidas = df_partidas[filter_1 & filter_2]

    # Obtain the partidas from the previous solution

    # Reset index
    df_partidas = df_partidas.reset_index(drop=True)

    return df_partidas

In [12]:
df_viajes, df_partidas = load_data("input_total/ult_version_20250606.xlsx")
df_viajes, df_partidas = preprocess_data(df_viajes, df_partidas)

In [13]:
# Select voyages
container_dimensions = (1350, 246, 259)
df_viajes_filtered = select_viajes(df_viajes)

In [14]:
# For each voyage, get the available partidas at the time of the voyage
df_partidas_filtered = select_initial_partidas(df_partidas)
df_partidas_filtered.sort_values(by=["FechaCargaContenedor"], ascending=False).head(5)

,CodigoViaje,FechaCargaContenedor,FechaEntradaAlmacen,Expedicion,Partida,PesoKg,LargoCm,AltoCm,AnchoCm,TipoPartida,Remontable,Destino,DatosClonados,Adr,DtLevanteExport,PuroVolumetrica,TratadaVOlumetrica,Cliente,Volumen
158,VBCN2501768,2025-06-05 11:46:04,2025-06-04 09:49:44,EBCN2521122,SBCN25047868,3,46.0,42.0,17.0,BULTO,0,MAH,DATOS NO CLONADOS,0,NaN,0,1,DSV AIR & SEA SAU,32844.0
10,VBCN2501781,2025-06-04 18:16:01,2025-06-04 17:21:18,EBCN2521296,SBCN25048394,100,120.0,150.0,80.0,PALET,0,PMI,DATOS NO CLONADOS,0,NaN,0,0,PALIBEX LOGISTICA S.L.,1440000.0
5,VBCN2501780,2025-06-04 18:15:38,2025-06-04 17:21:18,EBCN2521300,SBCN25048401,150,120.0,60.0,100.0,PALET,0,IBZ,DATOS NO CLONADOS,0,NaN,0,0,PALIBEX LOGISTICA S.L.,720000.0
9,VBCN2501781,2025-06-04 18:15:05,2025-06-04 17:21:18,EBCN2521287,SBCN25048383,220,120.0,220.0,100.0,PALET,0,PMI,DATOS NO CLONADOS,0,NaN,0,0,PALIBEX LOGISTICA S.L.,2640000.0
4,VBCN2501780,2025-06-04 18:14:51,2025-06-04 17:21:18,EBCN2521302,SBCN25048403,200,200.0,135.0,120.0,PALET,0,IBZ,DATOS NO CLONADOS,0,NaN,0,0,PALIBEX LOGISTICA S.L.,3240000.0


In [15]:
solucion = {}
total = len(df_viajes_filtered)
for index, row in df_viajes_filtered.iterrows():
    print(f"Processing voyage {index + 1}/{total}...")
    # Get the voyage code and destination
    codigo_viaje = row["CodigoViaje"]
    destinos = row["Destinos"]

    # Get the time of the voyage
    time_viaje = row["FechaCierreEstancia"]

    # Select partidas that are scheduled to depart at the time of the voyage for each destination
    df_partidas_destino = select_partidas_at_time_for_dest(df_partidas_filtered, time_viaje, destinos)
    # Save the partidas df in an excel file
    df_partidas_destino.to_excel(f"data/inputs/week_input/partidas_{codigo_viaje}.xlsx", index=False)

    # If there are partidas for the destination, calculate volumes
    if not df_partidas_destino.empty:
        # Calculate volumes for the selected partidas
        volumes = get_volumes(codigo_viaje, file_path=f"data/inputs/week_input/partidas_{codigo_viaje}.xlsx", save_path="data/outputs/week_output/")
        print(f"Voyage: {codigo_viaje}, Destination: {destinos}, Volumes: {volumes}")

    with open(f"data/outputs/week_output/output_{codigo_viaje}.json", "r") as f:
        # Load the solution from the output file
        solucion = json.load(f)
    # Update the partidas DataFrame with the volumes from the solution
    df_partidas_filtered = update_partidas_warehouse(df_partidas_filtered, solucion)


Processing voyage 1/5...


Generating solutions: 100%|██████████| 15000/15000 [17:38<00:00, 14.18it/s]


Solution 1 with score (69.31118807541571, 96.81752484191509, 1350) and 143 not loaded boxes:
Solution 2 with score (69.17136648843966, 95.2836495031617, 1344) and 140 not loaded boxes:
Solution 3 with score (68.5625579121514, 94.72869617585064, 1347) and 149 not loaded boxes:
Solution 4 with score (68.43327648205697, 94.38452273411623, 1346) and 131 not loaded boxes:
Solution 5 with score (68.35124787970317, 94.74284853959651, 1349) and 139 not loaded boxes:
Voyage: VBCN2501780, Destination: ['IBZ'], Volumes: (59.78447236606836, (56.51626423171139, 97.82083709725985, 1350), (69.31118807541571, 96.81752484191509, 1350), 143)
Processing voyage 2/5...


Generating solutions: 100%|██████████| 15000/15000 [18:57<00:00, 13.18it/s] 


Solution 1 with score (60.51560154812188, 95.60252935862692, 1350) and 107 not loaded boxes:
Solution 2 with score (60.46697801169346, 95.66726889491117, 1350) and 103 not loaded boxes:
Solution 3 with score (60.444656038151976, 95.66726889491117, 1350) and 103 not loaded boxes:
Solution 4 with score (60.400012091069, 95.66726889491117, 1350) and 103 not loaded boxes:
Solution 5 with score (60.33304617044455, 95.66726889491117, 1350) and 104 not loaded boxes:
Voyage: VBCN2501764, Destination: ['IBZ'], Volumes: (55.64288255552892, (59.47192023614788, 96.97982535380909, 1350), (60.51560154812188, 95.60252935862692, 1350), 107)
Processing voyage 3/5...


...

Solution 1 with score (80.16480708350628, 94.9286359530262, 1339) and 120 not loaded boxes:
Solution 2 with score (78.98243539706954, 94.36856368563686, 1338) and 121 not loaded boxes:
Solution 3 with score (78.7616861925805, 95.48449262270401, 1349) and 117 not loaded boxes:
Solution 4 with score (78.6845405219389, 95.53086419753086, 1345) and 118 not loaded boxes:
Solution 5 with score (78.64658735390442, 95.53297199638664, 1350) and 119 not loaded boxes:
Voyage: VBCN2501768, Destination: ['MAH'], Volumes: (73.88987006184719, (77.35256510866267, 95.90454682324601, 1350), (80.16480708350628, 94.9286359530262, 1339), 120)
Processing voyage 4/5...


Generating solutions: 100%|██████████| 15000/15000 [37:40<00:00,  6.64it/s] 


Solution 1 with score (79.997854997855, 93.24661246612466, 1348) and 493 not loaded boxes:
Solution 2 with score (79.86067135660632, 92.68172237277929, 1350) and 501 not loaded boxes:
Solution 3 with score (79.83775529303985, 93.1306835290575, 1350) and 492 not loaded boxes:
Solution 4 with score (79.83576375446295, 92.23065341764529, 1345) and 504 not loaded boxes:
Solution 5 with score (79.79462970519882, 92.65552544414332, 1350) and 487 not loaded boxes:
Voyage: VBCN2501779, Destination: ['PMI'], Volumes: (76.08740733667291, (76.96797029317355, 93.84884071062933, 1350), (79.997854997855, 93.24661246612466, 1348), 493)
Processing voyage 5/5...


Generating solutions: 100%|██████████| 15000/15000 [26:00<00:00,  9.61it/s]  


Solution 1 with score (61.52305383199692, 92.74947305028606, 1348) and 433 not loaded boxes:
Solution 2 with score (60.957094144086014, 92.67449563384523, 1349) and 438 not loaded boxes:
Solution 3 with score (60.42420818030574, 92.02950918398074, 1348) and 430 not loaded boxes:
Solution 4 with score (59.93352469775234, 93.66726889491117, 1340) and 434 not loaded boxes:
Solution 5 with score (59.780656382282395, 92.53538090936465, 1350) and 434 not loaded boxes:
Voyage: VBCN2501765, Destination: ['PMI'], Volumes: (54.85590800754893, (56.67041954846833, 96.62059620596206, 1350), (61.52305383199692, 92.74947305028606, 1348), 433)


In [18]:
outputs = os.listdir("data/outputs/week_output/")

for file in outputs:
    if file.endswith(".json"):
        with open(f"data/outputs/week_output/{file}", "r") as f:
            solucion = json.load(f)
        # Extract the voyage code from the filename
        codigo_viaje = file.split("_")[1].split(".")[0]
        # Evalutate the solution
        results = evaluate_solution(container_dimensions, solucion["solution"], f"data/inputs/week_input/partidas_{codigo_viaje}.xlsx")
        print(f"Results for voyage {codigo_viaje}:")
        print(json.dumps(results, indent=4))
        print("\n" + "="*50 + "\n")

Results for voyage VBCN2501764:
{
    "Volume Utilization": 0.6051560154812188,
    "Floor Utilization": 0.9560252935862692,
    "Max X": 1350,
    "Weight Total Deviation": 127.3573476233523,
    "Weight Horizontal Deviation": 119.87451445824463,
    "Not Loaded Box Count": 107
}


Results for voyage VBCN2501765:
{
    "Volume Utilization": 0.6152305383199692,
    "Floor Utilization": 0.9274947305028606,
    "Max X": 1348,
    "Weight Total Deviation": 37.595189040236484,
    "Weight Horizontal Deviation": 7.1415211047112885,
    "Not Loaded Box Count": 433
}


Results for voyage VBCN2501768:
{
    "Volume Utilization": 0.8016480708350627,
    "Floor Utilization": 0.949286359530262,
    "Max X": 1339,
    "Weight Total Deviation": 19.77653597068148,
    "Weight Horizontal Deviation": 3.6067509887516223,
    "Not Loaded Box Count": 120
}


Results for voyage VBCN2501779:
{
    "Volume Utilization": 0.79997854997855,
    "Floor Utilization": 0.9324661246612466,
    "Max X": 1348,
    "W

In [17]:
results

{'Volume Utilization': 0.6931118807541572,
 'Floor Utilization': 0.9681752484191508,
 'Max X': 1350,
 'Weight Total Deviation': 207.36685508621727,
 'Weight Horizontal Deviation': 204.4250059894497,
 'Not Loaded Box Count': 143}